## Q.1. チャネル入れ替え

画像を読み込み、BGRをRGBの順に入れ替えよ。

画像の赤成分を取り出すには、以下のコードで可能。
cv2.imread()関数ではチャネルがBGRの順になることに注意！
これで変数redにimori.jpgの赤成分のみが入る。

```python
import cv2
img = cv2.imread("imori.jpg")
red = img[:, :, 2].copy()
```

In [9]:
import cv2

# 画像の読み込み
img = cv2.imread("imori.jpg")

# それぞれの値を格納する
Red = img[:,:,2].copy()
Blue = img[:,:,0].copy()
Green = img[:,:,1].copy()

# サイズをコピーする
img_RGB = img

# 付け替える
img_RGB[:,:,0] = Red
img_RGB[:,:,1] = Green
img_RGB[:,:,2] = Blue

# 保存して確認
cv2.imwrite("training_IMG/training_01.png",img_RGB)

True

In [10]:
%reset -f

## Q.2. グレースケール化

画像をグレースケールにせよ。
グレースケールとは、画像の輝度表現方法の一種であり下式で計算される。

Y = 0.2126 R + 0.7152 G + 0.0722 B

In [11]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg")

# グレースケール化
img_Y = 0.2126*img[:,:,2] + 0.7152*img[:,:,1] + 0.0722*img[:,:,0]

# 変数の型を unit8 にして次元をまとめる
img_Y = np.expand_dims(img_Y.astype(np.uint8), axis=-1)

# 保存して確認する
cv2.imwrite("training_IMG/training_02.png",img_Y)

True

In [12]:
%reset -f

## Q.3. 二値化

画像を二値化せよ。
二値化とは、画像を黒と白の二値で表現する方法である。
ここでは、グレースケールにおいて閾値を128に設定し、下式で二値化する。

```bash
y = { 0 (if y < 128)
     255 (else) }
```

In [13]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg")

# グレースケール化
img_Y = 0.3*img[:,:,2] + 0.59*img[:,:,1] + 0.11*img[:,:,0]
img_Y = np.expand_dims(img_Y.astype(np.uint8), axis=-1)

# ２値化
img_Y[img_Y < 128] = 0
img_Y[img_Y >= 128] = 255

# 保存して確認する
cv2.imwrite("training_IMG/training_03.png",img_Y)

True

## Q.4. 大津の二値化

大津の二値化を実装せよ。
大津の二値化とは判別分析法と呼ばれ、二値化における分離の閾値を自動決定する手法である。
これは**クラス内分散**と**クラス間分散**の比から計算される。

グレースケールの輝度値（ピクセルの値）のヒストグラムはこうなる。

```python
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('assets/imori.jpg')
gray = 0.2126 * img[..., 2] + 0.7152 * img[..., 1] + 0.0722 * img[..., 0]
plt.hist(gray.ravel(), bins=255, rwidth=0.8, range=(0, 255))
plt.xlabel('value')
plt.ylabel('appearance')
plt.show()
```

<img src="assets/histogram-Gray.jpg" width=400>

二値化はある値を境界にして、０か１にする方法だけど、

- 閾値t未満をクラス0, t以上をクラス1とする。
- w0, w1 ... 閾値tにより分離された各クラスの画素数の割合 (w0 + w1 = 1を満たす)
- S0^2, S1^2 ... 各クラスの画素値の分散
- M0, M1 ... 各クラスの画素値の平均値

<img src="assets/histogram-Gray-1.png" width=400>

とすると、

<img src="assets/otsu_binary_1.png" width=500>

となり、分離度Xは次式で定義される。

<img src="assets/otsu_binary_2.png" width=300>

<!--
```bash
クラス内分散 Sw^2 = w0 * S0^2 + w1 * S1^2
クラス間分散 Sb^2 = w0 * (M0 - Mt)^2 + w1 * (M1 - Mt)^2 = w0 * w1 * (M0 - M1) ^2
画像全体の画素の分散 St^2 = Sw^2 + Sb^2 = (const)
以上より、分離度は次式で定義される。
分離度 X = Sb^2 / Sw^2 = Sb^2 / (St^2 - Sb^2)
```
-->

となるので、

<img src="assets/otsu_binary_3.png" width=300>

<!--
```bash
argmax_{t} X = argmax_{t} Sb^2
```
-->


となる。すなわち、Sb^2 =  w0 * w1 * (M0 - M1) ^2 が最大となる、閾値tを二値化の閾値とすれば良い。


In [14]:
import cv2
import numpy as np

img = cv2.imread("imori.jpg")

# グレースケール化
img_Y = 0.2126*img[:,:,2] + 0.7152*img[:,:,1] + 0.0722*img[:,:,0]
img_Y = np.expand_dims(img_Y.astype(np.uint8), axis=-1)

# 大津の２値化のための閾値探し
max_Sb2 = 0
max_t = 0
H, W, C = img.shape
for t in range(0,256):
    # 閾値以上か未満で2値化
    v0 = img_Y[np.where(img_Y < t)]
    v1 = img_Y[np.where(img_Y >= t)]

    # 各クラスの画素数の割合を算出
    w0 = len(v0) / (H*W)
    w1 = len(v1) / (H*W)

    # 各クラスの画素数の平均を算出
    m0 = np.mean(v0) if len(v0) > 0 else 0
    m1 = np.mean(v1) if len(v0) > 0 else 0

    # 判定式
    tmp_Sb2 = w0*w1*(m0-m1)**2
    if tmp_Sb2 > max_Sb2:
        max_Sb2 = tmp_Sb2
        max_t = t

print("最適な閾値は",max_t)

# 最適な閾値で２値化
img_Y[img_Y < max_t] = 0
img_Y[img_Y >= max_t] = 255

# 保存して確認する
cv2.imwrite("training_IMG/training_04.png",img_Y)

最適な閾値は 127


/Users/graduser/.anyenv/envs/pyenv/versions/miniforge3-22.11.1-4/envs/100knock/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/graduser/.anyenv/envs/pyenv/versions/miniforge3-22.11.1-4/envs/100knock/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


True